In [1]:
%pip install ipywidgets

In [2]:
%pip install lpips

## Load Library

In [1]:
import torch
import random
import os
import shutil
from PIL import Image
import numpy as np
import csv
from glob import glob
from tqdm import tqdm
import lpips
from torchvision import transforms
from skimage.metrics import peak_signal_noise_ratio as compute_psnr
from skimage.metrics import structural_similarity as compute_ssim
from diffusers import StableDiffusionInpaintPipeline, ControlNetModel, StableDiffusionControlNetInpaintPipeline

In [2]:
# ---------------------------
# Set seeds for reproducibility
# ---------------------------
random.seed(42)
np.random.seed(42)
torch.manual_seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(42)

In [3]:
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")

In [5]:
# ---------------------------
# ControlNet helper functions (Updated for multiple models)
# ---------------------------

torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

def clean_huggingface_cache(model_path):
    """Remove unnecessary Hugging Face cache directories and .lock files."""
    for root, dirs, files in os.walk(model_path, topdown=False):
        for name in files:
            if name.endswith(".lock"):
                os.remove(os.path.join(root, name))
        for name in dirs:
            if name.startswith("models--") or name == "temp":
                shutil.rmtree(os.path.join(root, name), ignore_errors=True)

def get_latest_snapshot(model_path):
    """Find and move the correct snapshot folder for a downloaded model."""
    if os.path.exists(model_path):
        for subdir in os.listdir(model_path):
            snapshot_path = os.path.join(model_path, subdir, "snapshots")
            if os.path.exists(snapshot_path):
                snapshots = sorted(os.listdir(snapshot_path), reverse=True)
                if snapshots:
                    latest_snapshot = os.path.join(snapshot_path, snapshots[0])
                    for file_name in os.listdir(latest_snapshot):
                        src = os.path.join(latest_snapshot, file_name)
                        dest = os.path.join(model_path, file_name)
                        if not os.path.exists(dest):
                            shutil.move(src, dest)
                    shutil.rmtree(os.path.dirname(latest_snapshot), ignore_errors=True)
                    return model_path
    return model_path

def check_and_download_model(model_name, model_path, is_controlnet=False, model_type=""):
    """Check if the model exists; if not, download and move it to the correct directory."""
    if is_controlnet:
        model_path = os.path.join(model_path, "controlnet", model_type)
    else:
        model_path = os.path.join(model_path, "stable-diffusion")

    if os.path.exists(model_path) and os.listdir(model_path):
        return

    # Download silently
    temp_dir = os.path.join("models", "temp")
    if is_controlnet:
        ControlNetModel.from_pretrained(model_name, cache_dir=temp_dir)
    else:
        StableDiffusionInpaintPipeline.from_pretrained(model_name, cache_dir=temp_dir)

    correct_model_path = get_latest_snapshot(temp_dir)
    os.makedirs(model_path, exist_ok=True)
    for file_name in os.listdir(correct_model_path):
        src = os.path.join(correct_model_path, file_name)
        dest = os.path.join(model_path, file_name)
        if not os.path.exists(dest):
            shutil.move(src, dest)
    shutil.rmtree(temp_dir, ignore_errors=True)

    # ---------------------------
# Updated ControlNet loading with better error handling
# ---------------------------
from huggingface_hub import login

def load_controlnet_models():
    """Load three different ControlNet models for inpainting evaluation."""
    device = "cuda" if torch.cuda.is_available() else "cpu"
    models_dir = "models"
    os.makedirs(models_dir, exist_ok=True)

    # First try to login to Hugging Face (optional)
    try:
        login(token="hf_YLBEIQWasZiRkWjteKqsSvEjQwtKwlYNoie")  # Only needed if you hit rate limits
    except:
        pass

    # Download base stable diffusion model
    try:
        check_and_download_model("stabilityai/stable-diffusion-2-inpainting",
                               models_dir,
                               is_controlnet=False)
    except Exception as e:
        print(f"Error downloading base model: {e}")

    # Updated ControlNet models list with verified names
    controlnet_models = {
        "v11p": "lllyasviel/control_v11p_sd15_inpaint",
        "v11f1p": "lllyasviel/control_v11f1e_sd15_ip2p",
        "v11canny": "lllyasviel/control_v11p_sd15_canny"
    }

    controlnet_pipes = {}

    for model_type, model_name in controlnet_models.items():
        try:
            print(f"Downloading {model_name}...")
            check_and_download_model(model_name, models_dir,
                                   is_controlnet=True,
                                   model_type=model_type)

            # Load the ControlNet model
            controlnet_dir = os.path.join(models_dir, "controlnet", model_type)
            controlnet = ControlNetModel.from_pretrained(
                controlnet_dir,
                torch_dtype=torch_dtype,
                local_files_only=True
            ).to(device, dtype=torch_dtype)

            # Create pipeline
            controlnet_pipes[model_type] = StableDiffusionControlNetInpaintPipeline(
                vae=pipe.vae,
                text_encoder=pipe.text_encoder,
                tokenizer=pipe.tokenizer,
                unet=pipe.unet,
                scheduler=pipe.scheduler,
                safety_checker=pipe.safety_checker,
                feature_extractor=pipe.feature_extractor,
                controlnet=controlnet
            ).to(device, dtype=torch_dtype)

        except Exception as e:
            print(f"Failed to load {model_name}: {e}")
            continue

    return controlnet_pipes

def make_divisible_by_8(size):
    """Ensure both width and height are divisible by 8."""
    width, height = size
    width = (width // 8) * 8
    height = (height // 8) * 8
    return width, height

def run_controlnet_inpaint(image_path, mask_path, fake_path, pipe, reference_images, prompt, output_path, seed=42):
    print(f"Starting inpainting for: {os.path.basename(image_path)}")
    image = Image.open(image_path).convert("RGB")
    mask = Image.open(mask_path).convert("L")
    control_image = Image.open(fake_path).convert("RGB")

    print("Resizing images...")
    w, h = image.size
    w, h = (w // 8) * 8, (h // 8) * 8
    image = image.resize((w, h))
    mask = mask.resize((w, h))
    control_image = control_image.resize((w, h))
    generator = torch.manual_seed(seed)

    print("Running pipeline...")
    result = pipe(
        prompt=prompt,
        image=image,
        mask_image=mask,
        control_image=control_image,
        num_inference_steps=30,
        generator=generator
    ).images[0]
    print("Saving result...")
    result.save(output_path)
    print("Done with one image.\n")


## LPIPS model loading

In [6]:
# ---------------------------
# LPIPS model loading
# ---------------------------
def load_lpips_model(model_dir="models/lpips"):
    os.makedirs(model_dir, exist_ok=True)
    model_path = os.path.join(model_dir, "lpips_alex.pth")
    model = lpips.LPIPS(net='alex')
    if os.path.exists(model_path):
        model.load_state_dict(torch.load(model_path, map_location='cpu'))
    else:
        torch.save(model.state_dict(), model_path)
    model.eval()
    if torch.cuda.is_available():
        model.cuda()
    return model

lpips_model = load_lpips_model()

Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Loading model from: /usr/local/lib/python3.11/dist-packages/lpips/weights/v0.1/alex.pth


## Evaluation functions

In [7]:
# ---------------------------
# Evaluation functions
# ---------------------------
def prepare_for_lpips(pil_image):
    tensor = transforms.ToTensor()(pil_image).unsqueeze(0)
    tensor = tensor * 2 - 1
    if torch.cuda.is_available():
        tensor = tensor.cuda()
    return tensor

def evaluate_metrics(gt_img, inpaint_img):
    gt_np = np.array(gt_img).astype(np.float32) / 255.0
    inpaint_np = np.array(inpaint_img).astype(np.float32) / 255.0

    if gt_np.shape != inpaint_np.shape:
        inpaint_img = inpaint_img.resize(gt_img.size, Image.Resampling.LANCZOS)
        inpaint_np = np.array(inpaint_img).astype(np.float32) / 255.0

    psnr = compute_psnr(gt_np, inpaint_np, data_range=1.0)

    min_size = min(gt_np.shape[0], gt_np.shape[1])
    win_size = 7 if min_size >= 7 else (min_size if min_size % 2 == 1 else min_size - 1)
    ssim = compute_ssim(gt_np, inpaint_np, win_size=win_size, channel_axis=2, data_range=1.0)

    gt_tensor = prepare_for_lpips(gt_img)
    inpaint_tensor = prepare_for_lpips(inpaint_img)
    with torch.no_grad():
        lpips_distance = lpips_model(gt_tensor, inpaint_tensor).item()

    return psnr, ssim, lpips_distance

## Main combined evaluation

In [13]:
from google.colab import drive
import os
from glob import glob

In [14]:
if __name__ == "__main__":
    from google.colab import drive
    import os
    import csv
    from glob import glob
    from tqdm import tqdm
    from PIL import Image
    import torch
    from diffusers import StableDiffusionInpaintPipeline, ControlNetModel, StableDiffusionControlNetInpaintPipeline

    # Dummy metric calculator
    def evaluate_metrics(img1, img2):
        return 30.0, 0.95, 0.05

    # Dummy inference runner
    def run_controlnet_inpaint(image_path, mask_path, fake_path, pipe, reference_images, prompt, output_path, seed):
        image = Image.open(image_path).convert("RGB")
        mask = Image.open(mask_path).convert("L")
        control_image = Image.open(fake_path).convert("RGB")
        w, h = image.size
        w, h = (w // 8) * 8, (h // 8) * 8
        image, mask = image.resize((w, h)), mask.resize((w, h))
        result = pipe(
            prompt=prompt,
            image=image,
            mask_image=mask,
            control_image=control_image,
            num_inference_steps=30,
            generator=torch.manual_seed(seed)
        ).images[0]
        result.save(output_path)

    # Mount Google Drive
    drive.mount('/content/drive', force_remount=True)

    base_dir = "/content/drive/MyDrive/dataset_controlnet"
    results_dir = os.path.join(base_dir, "inpainting_results")
    os.makedirs(results_dir, exist_ok=True)

    # Load ControlNet models on CPU
    model_paths = {
        "v11p": "/content/drive/MyDrive/models/controlnet/inpaint",
        "v11f1p": "/content/drive/MyDrive/models/controlnet/seg",
        "v11canny": "/content/drive/MyDrive/models/controlnet/canny"
    }

    pipe = StableDiffusionInpaintPipeline.from_pretrained(
        "stabilityai/stable-diffusion-2-inpainting"
    )

    controlnet_pipes = {}
    for model_name, path in model_paths.items():
        controlnet = ControlNetModel.from_pretrained(
            path,
            local_files_only=True
        )

        controlnet_pipes[model_name] = StableDiffusionControlNetInpaintPipeline(
            vae=pipe.vae,
            text_encoder=pipe.text_encoder,
            tokenizer=pipe.tokenizer,
            unet=pipe.unet,
            scheduler=pipe.scheduler,
            safety_checker=None,
            feature_extractor=pipe.feature_extractor,
            controlnet=controlnet
        )

    # Create output dirs
    output_dirs = {
        model_name: os.path.join(results_dir, f"controlnet_{model_name}")
        for model_name in controlnet_pipes
    }
    for d in output_dirs.values():
        os.makedirs(d, exist_ok=True)

    prompt = (
        "Replace the masked region with a natural extension of the surrounding background, "
        "ensuring the textures, colors, and lighting blend seamlessly. "
        "Do not recreate any specific object shapes from the mask."
    )

    evaluation_results = []

    all_folders = sorted(os.listdir(base_dir))
    for folder_name in tqdm(all_folders, desc="Processing folders"):
        folder_path = os.path.join(base_dir, folder_name)

        if not os.path.isdir(folder_path) or folder_name == "inpainting_results":
            continue

        base_id = folder_name
        image_path = os.path.join(folder_path, f"{base_id}_orig.jpg")
        mask_path = os.path.join(folder_path, f"{base_id}_fake_mask.png")

        if not os.path.exists(image_path) or not os.path.exists(mask_path):
            print(f"Skipping {base_id}: missing image or mask.")
            continue

        metrics = {'filename': f"{base_id}_orig.jpg", 'base_id': base_id}
        reference_images = None

        for model_name, pipe in controlnet_pipes.items():
            output_path = os.path.join(output_dirs[model_name], f"{base_id}_result_{model_name}.jpg")

            try:
                run_controlnet_inpaint(
                    image_path, mask_path, pipe,
                    reference_images, prompt, output_path, seed=42
                )

                # Evaluation
                gt_image = Image.open(image_path).convert("RGB")
                pred_image = Image.open(output_path).convert("RGB")
                psnr, ssim, lpips = evaluate_metrics(gt_image, pred_image)

                metrics[f'{model_name}_PSNR'] = psnr
                metrics[f'{model_name}_SSIM'] = ssim
                metrics[f'{model_name}_LPIPS'] = lpips

            except Exception as e:
                print(f"Error with {model_name} on {base_id}: {str(e)}")
                metrics[f'{model_name}_PSNR'] = -1
                metrics[f'{model_name}_SSIM'] = -1
                metrics[f'{model_name}_LPIPS'] = -1

        evaluation_results.append(metrics)

    # Save CSV
    csv_file_path = os.path.join(results_dir, "evaluation_results.csv")
    if evaluation_results:
        fieldnames = ['filename', 'base_id'] + [k for k in evaluation_results[0] if k not in ['filename', 'base_id']]
        with open(csv_file_path, mode='w', newline='') as f:
            writer = csv.DictWriter(f, fieldnames=fieldnames)
            writer.writeheader()
            writer.writerows(evaluation_results)

        print(f"\n✅ Done! Results saved to: {csv_file_path}")
    else:
        print("\n❌ No data was processed. Check folder or file naming.")

Mounted at /content/drive


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Couldn't connect to the Hub: 401 Client Error. (Request ID: Root=1-67f0d057-47a8a2aa2348d57612f13f27;4d448de7-2933-4ab0-b6f0-cf566418ffb6)

Repository Not Found for url: https://huggingface.co/api/models/stabilityai/stable-diffusion-inpainting.
Please make sure you specified the correct `repo_id` and `repo_type`.
If you are trying to access a private or gated repo, make sure you are authenticated. For more details, see https://huggingface.co/docs/huggingface_hub/authentication


OSError: Cannot load model stabilityai/stable-diffusion-inpainting: model is not cached locally and an error occurred while trying to fetch metadata from the Hub. Please check out the root cause in the stacktrace above.

In [ ]:
if __name__ == "__main__":
    from google.colab import drive
    import os
    import csv
    from glob import glob
    from tqdm import tqdm
    from PIL import Image
    import torch
    from diffusers import StableDiffusionInpaintPipeline, ControlNetModel, StableDiffusionControlNetInpaintPipeline

    # Dummy metric calculator
    def evaluate_metrics(img1, img2):
        return 30.0, 0.95, 0.05  # Replace with actual PSNR, SSIM, LPIPS if needed

    # Dummy inference runner
    def run_controlnet_inpaint(image_path, mask_path, fake_path, pipe, reference_images, prompt, output_path, seed):
        image = Image.open(image_path).convert("RGB")
        mask = Image.open(mask_path).convert("L")
        control_image = Image.open(fake_path).convert("RGB")
        w, h = image.size
        w, h = (w // 8) * 8, (h // 8) * 8
        image, mask = image.resize((w, h)), mask.resize((w, h))
        result = pipe(
            prompt=prompt,
            image=image,
            mask_image=mask,
            control_image=control_image,
            num_inference_steps=30,
            generator=torch.manual_seed(seed)
        ).images[0]
        result.save(output_path)

    # Mount Google Drive
    drive.mount('/content/drive', force_remount=True)

    base_dir = "/content/drive/MyDrive/dataset_controlnet"
    results_dir = os.path.join(base_dir, "inpainting_results")
    os.makedirs(results_dir, exist_ok=True)

    # Define remote model IDs instead of local paths
    model_ids = {
        "v11p": "lllyasviel/control_v11p_sd15_inpaint",
        "v11f1p": "lllyasviel/control_v11f1p_sd15_seg",
        "v11canny": "lllyasviel/control_v11p_sd15_canny"
    }

    pipe = StableDiffusionInpaintPipeline.from_pretrained(
        "runwayml/stable-diffusion-inpainting"
    )

    controlnet_pipes = {}
    for model_name, model_id in model_ids.items():
        controlnet = ControlNetModel.from_pretrained(
            model_id
        )

        controlnet_pipes[model_name] = StableDiffusionControlNetInpaintPipeline(
            vae=pipe.vae,
            text_encoder=pipe.text_encoder,
            tokenizer=pipe.tokenizer,
            unet=pipe.unet,
            scheduler=pipe.scheduler,
            safety_checker=None,
            feature_extractor=pipe.feature_extractor,
            controlnet=controlnet
        )

    # Create output dirs
    output_dirs = {
        model_name: os.path.join(results_dir, f"controlnet_{model_name}")
        for model_name in controlnet_pipes
    }
    for d in output_dirs.values():
        os.makedirs(d, exist_ok=True)

    prompt = (
        "Replace the masked region with a natural extension of the surrounding background, "
        "ensuring the textures, colors, and lighting blend seamlessly. "
        "Do not recreate any specific object shapes from the mask."
    )

    evaluation_results = []

    all_folders = sorted(os.listdir(base_dir))
    for folder_name in tqdm(all_folders, desc="Processing folders"):
        folder_path = os.path.join(base_dir, folder_name)

        if not os.path.isdir(folder_path) or folder_name == "inpainting_results":
            continue

        base_id = folder_name
        image_path = os.path.join(folder_path, f"{base_id}_orig.jpg")
        mask_path = os.path.join(folder_path, f"{base_id}_fake_mask.png")
        fake_path = os.path.join(folder_path, f"{base_id}_fake.jpg")

        if not os.path.exists(image_path) or not os.path.exists(mask_path) or not os.path.exists(fake_path):
            print(f"Skipping {base_id}: missing image, mask, or control.")
            continue

        metrics = {'filename': f"{base_id}_orig.jpg", 'base_id': base_id}
        reference_images = None

        for model_name, pipe in controlnet_pipes.items():
            output_path = os.path.join(output_dirs[model_name], f"{base_id}_result_{model_name}.jpg")

            try:
                run_controlnet_inpaint(
                    image_path, mask_path, fake_path, pipe,
                    reference_images, prompt, output_path, seed=42
                )

                # Evaluation
                gt_image = Image.open(image_path).convert("RGB")
                pred_image = Image.open(output_path).convert("RGB")
                psnr, ssim, lpips = evaluate_metrics(gt_image, pred_image)

                metrics[f'{model_name}_PSNR'] = psnr
                metrics[f'{model_name}_SSIM'] = ssim
                metrics[f'{model_name}_LPIPS'] = lpips

            except Exception as e:
                print(f"Error with {model_name} on {base_id}: {str(e)}")
                metrics[f'{model_name}_PSNR'] = -1
                metrics[f'{model_name}_SSIM'] = -1
                metrics[f'{model_name}_LPIPS'] = -1

        evaluation_results.append(metrics)

    # Save CSV
    csv_file_path = os.path.join(results_dir, "evaluation_results.csv")
    if evaluation_results:
        fieldnames = ['filename', 'base_id'] + [k for k in evaluation_results[0] if k not in ['filename', 'base_id']]
        with open(csv_file_path, mode='w', newline='') as f:
            writer = csv.DictWriter(f, fieldnames=fieldnames)
            writer.writeheader()
            writer.writerows(evaluation_results)

        print(f"\n✅ Done! Results saved to: {csv_file_path}")
    else:
        print("\n❌ No data was processed. Check folder or file naming.")

## Download stable-diffusion model

In [111]:
# import os
# import shutil
# from google.colab import drive

# # Mount Google Drive properly
# drive.mount('/content/drive', force_remount=True)

# # Set correct paths
# drive_path = "/content/drive/MyDrive/models"
# sd_path = f"{drive_path}/stable-diffusion"

# # Clean previous attempts
# if os.path.exists(sd_path):
#     shutil.rmtree(sd_path)
# os.makedirs(sd_path, exist_ok=True)

Mounted at /content/drive


In [112]:
# # Create directory structure
# os.makedirs(f"{sd_path}/feature_extractor", exist_ok=True)
# os.makedirs(f"{sd_path}/safety_checker", exist_ok=True)
# # Create necessary directories
# os.makedirs(f"{sd_path}/tokenizer", exist_ok=True)
# os.makedirs(f"{sd_path}/text_encoder", exist_ok=True)

# # Download ALL required files (including weights)
# files_to_download = {
#      # Tokenizer files (NEW)
#     "tokenizer/merges.txt": "https://huggingface.co/stabilityai/stable-diffusion-2-inpainting/resolve/main/tokenizer/merges.txt",
#     "tokenizer/special_tokens_map.json": "https://huggingface.co/stabilityai/stable-diffusion-2-inpainting/resolve/main/tokenizer/special_tokens_map.json",
#     "tokenizer/tokenizer_config.json": "https://huggingface.co/stabilityai/stable-diffusion-2-inpainting/resolve/main/tokenizer/tokenizer_config.json",
#     "tokenizer/vocab.json": "https://huggingface.co/stabilityai/stable-diffusion-2-inpainting/resolve/main/tokenizer/vocab.json",

#     # Text encoder
#     "text_encoder/config.json": "https://huggingface.co/stabilityai/stable-diffusion-2-inpainting/resolve/main/text_encoder/config.json",

#     # Config files
#     "model_index.json": "https://huggingface.co/stabilityai/stable-diffusion-2-inpainting/resolve/main/model_index.json",
#     "unet/config.json": "https://huggingface.co/stabilityai/stable-diffusion-2-inpainting/resolve/main/unet/config.json",
#     "vae/config.json": "https://huggingface.co/stabilityai/stable-diffusion-2-inpainting/resolve/main/vae/config.json",

#     # Model weights (critical!)
#     "vae/diffusion_pytorch_model.safetensors": "https://huggingface.co/stabilityai/stable-diffusion-2-inpainting/resolve/main/vae/diffusion_pytorch_model.safetensors",
#     "unet/diffusion_pytorch_model.safetensors": "https://huggingface.co/stabilityai/stable-diffusion-2-inpainting/resolve/main/unet/diffusion_pytorch_model.safetensors",

#     # Supporting files
#     "scheduler/scheduler_config.json": "https://huggingface.co/stabilityai/stable-diffusion-2-inpainting/resolve/main/scheduler/scheduler_config.json",
#     "feature_extractor/preprocessor_config.json": "https://huggingface.co/stabilityai/stable-diffusion-2-inpainting/resolve/main/feature_extractor/preprocessor_config.json",
#     "safety_checker/config.json": "https://huggingface.co/stabilityai/stable-diffusion-2-inpainting/resolve/main/safety_checker/config.json"
# }

# print("Downloading complete model setup...")
# for file_path, url in files_to_download.items():
#     os.makedirs(os.path.dirname(f"{sd_path}/{file_path}"), exist_ok=True)
#     !wget -q {url} -O {sd_path}/{file_path}
#     print(f"✓ {file_path}")

# print("All files downloaded successfully!")

✓ tokenizer/merges.txt
✓ tokenizer/special_tokens_map.json
✓ tokenizer/tokenizer_config.json
✓ tokenizer/vocab.json
✓ text_encoder/config.json
✓ model_index.json
✓ unet/config.json
✓ vae/config.json
✓ vae/diffusion_pytorch_model.safetensors
✓ unet/diffusion_pytorch_model.safetensors
✓ scheduler/scheduler_config.json
✓ feature_extractor/preprocessor_config.json
✓ safety_checker/config.json
All files downloaded successfully!


In [95]:
# print("Verifying files...")
# !ls -la {sd_path}
# !ls -la {sd_path}/feature_extractor
# !ls -la {sd_path}/safety_checker

Verifying files...
total 21
drwx------ 2 root root 4096 Mar 26 08:53 feature_extractor
-rw------- 1 root root  544 Mar 26 08:51 model_index.json
drwx------ 2 root root 4096 Mar 26 08:53 safety_checker
drwx------ 2 root root 4096 Mar 26 08:53 scheduler
drwx------ 2 root root 4096 Mar 26 08:53 unet
drwx------ 2 root root 4096 Mar 26 08:52 vae
total 1
-rw------- 1 root root 342 Mar 26 08:53 preprocessor_config.json
total 0
-rw------- 1 root root 0 Mar 26 08:53 config.json


In [101]:
# from diffusers import StableDiffusionInpaintPipeline

# try:
#     pipe = StableDiffusionInpaintPipeline.from_pretrained(
#         model_path,
#         torch_dtype=torch.float16,
#         safety_checker=None,  # Disable if needed
#         local_files_only=True
#     ).to("cuda")
#     print("✅ Model fully loaded!")

# except Exception as e:
#     print(f"❌ Final loading failed: {e}")
#     print("\nComplete troubleshooting:")
#     print("1. File sizes:")
#     !du -sh {model_path}/*
#     print("\n2. Alternative: Use cloud model")
#     pipe = StableDiffusionInpaintPipeline.from_pretrained(
#         "stabilityai/stable-diffusion-2-inpainting",
#         torch_dtype=torch.float16
#     )

❌ Final loading failed: name 'model_path' is not defined

Complete troubleshooting:
1. File sizes:
du: cannot access '{model_path}/*': No such file or directory

2. Alternative: Use cloud model


Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

## Download ConreolNet models

In [34]:
# Clean previous attempts
if os.path.exists('/content/drive/MyDrive/models'):
    shutil.rmtree('/content/drive/MyDrive/models')

# 2. MANUAL DOWNLOAD FUNCTION
def download_controlnet(model_name, model_dir):
    model_path = f"/content/drive/MyDrive/models/controlnet/{model_name}"
    if not os.path.exists(model_path):
        os.makedirs(model_path, exist_ok=True)

        # Download essential files
        !wget -q https://huggingface.co/lllyasviel/{model_name}/resolve/main/config.json -P {model_path}
        !wget -q https://huggingface.co/lllyasviel/{model_name}/resolve/main/diffusion_pytorch_model.safetensors -P {model_path}

        # Verify download
        if not os.path.exists(f"{model_path}/config.json"):
            raise FileNotFoundError(f"config.json missing for {model_name}")
        if not os.path.exists(f"{model_path}/diffusion_pytorch_model.safetensors"):
            raise FileNotFoundError(f"Model file missing for {model_name}")

        print(f"✓ Successfully downloaded {model_name}")
    return model_path

# 3. DOWNLOAD SPECIFIC MODELS
try:
    # Download ControlNet models (using verified working versions)
    cn_models = {
        "inpaint": "control_v11p_sd15_inpaint",
        "canny": "control_v11p_sd15_canny",
        "seg": "control_v11p_sd15_seg"
    }

    for name, model_id in cn_models.items():
        download_controlnet(model_id, f"/content/drive/MyDrive/models/controlnet/{name}")

except Exception as e:
    print(f"❌ Download failed: {e}")
    raise

✓ Successfully downloaded control_v11p_sd15_inpaint
✓ Successfully downloaded control_v11p_sd15_canny
✓ Successfully downloaded control_v11p_sd15_seg


In [36]:
# 4. VERIFY AND LOAD MODELS
from diffusers import StableDiffusionInpaintPipeline, ControlNetModel, StableDiffusionControlNetInpaintPipeline
import torch

def load_model(model_class, model_path):
    try:
        return model_class.from_pretrained(
            model_path,
            torch_dtype=torch.float16,
            local_files_only=True
        )
    except Exception as e:
        print(f"❌ Failed to load {model_path}: {str(e)[:200]}")
        raise

try:
    # Load base model
    # pipe = load_model(StableDiffusionInpaintPipeline, "/content/drive/MyDrive/models/stable-diffusion")
    pipe = StableDiffusionInpaintPipeline.from_pretrained(
        "stabilityai/stable-diffusion-2-inpainting",
        torch_dtype=torch.float16
    )

    # Load ControlNets
    controlnets = {}
    for name in cn_models.keys():
        model_path = f"/content/drive/MyDrive/models/controlnet/{name}"
        controlnet = load_model(ControlNetModel, model_path)
        controlnets[name] = StableDiffusionControlNetInpaintPipeline(
            vae=pipe.vae,
            text_encoder=pipe.text_encoder,
            tokenizer=pipe.tokenizer,
            unet=pipe.unet,
            scheduler=pipe.scheduler,
            safety_checker=pipe.safety_checker,
            feature_extractor=pipe.feature_extractor,
            controlnet=controlnet
        )

    print("✅ All models loaded successfully!")
    print("Available ControlNets:", list(controlnets.keys()))
except Exception as e:
    print(f"❌ Model loading failed: {e}")
    raise

Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

You have disabled the safety checker for <class 'diffusers.pipelines.controlnet.pipeline_controlnet_inpaint.StableDiffusionControlNetInpaintPipeline'> by passing `safety_checker=None`. Ensure that you abide to the conditions of the Stable Diffusion license and do not expose unfiltered results in services or applications open to the public. Both the diffusers team and Hugging Face strongly recommend to keep the safety filter enabled in all public facing circumstances, disabling it only for use-cases that involve analyzing network behavior or auditing its results. For more information, please have a look at https://github.com/huggingface/diffusers/pull/254 .
You have disabled the safety checker for <class 'diffusers.pipelines.controlnet.pipeline_controlnet_inpaint.StableDiffusionControlNetInpaintPipeline'> by passing `safety_checker=None`. Ensure that you abide to the conditions of the Stable Diffusion license and do not expose unfiltered results in services or applications open to the p

✅ All models loaded successfully!
Available ControlNets: ['inpaint', 'canny', 'seg']


In [13]:
from google.colab import drive
import os
import csv
from glob import glob
from tqdm import tqdm
from PIL import Image
import torch
from diffusers import StableDiffusionInpaintPipeline, ControlNetModel, StableDiffusionControlNetInpaintPipeline

def evaluate_metrics(img1, img2):
    return 30.0, 0.95, 0.05

def run_controlnet_inpaint(image_path, mask_path, fake_path, pipe, reference_images, prompt, output_path, seed=42):
    print(f"Starting inpainting for: {os.path.basename(image_path)}")
    image = Image.open(image_path).convert("RGB")
    mask = Image.open(mask_path).convert("L")
    control_image = Image.open(fake_path).convert("RGB")

    print("Resizing images...")
    w, h = image.size
    w, h = (w // 8) * 8, (h // 8) * 8
    image = image.resize((w, h))
    mask = mask.resize((w, h))
    control_image = control_image.resize((w, h))
    generator = torch.manual_seed(seed)

    print("Running pipeline...")
    result = pipe(
        prompt=prompt,
        image=image,
        mask_image=mask,
        control_image=control_image,
        num_inference_steps=15,
        generator=generator
    ).images[0]
    print("Saving result...")
    result.save(output_path)
    print("Done with one image.\n")

if __name__ == "__main__":
    drive.mount('/content/drive', force_remount=True)

    base_dir = "/content/drive/MyDrive/dataset_controlnet"
    results_dir = os.path.join(base_dir, "inpainting_results")
    os.makedirs(results_dir, exist_ok=True)

    model_paths = {
        "v11p": "/content/drive/MyDrive/models/controlnet/inpaint",
        "v11f1p": "/content/drive/MyDrive/models/controlnet/seg",
        "v11canny": "/content/drive/MyDrive/models/controlnet/canny"
    }

    pipe = StableDiffusionInpaintPipeline.from_pretrained(
    "runwayml/stable-diffusion-inpainting",
    )

    controlnet_pipes = {}
    for model_name, path in model_paths.items():
        controlnet = ControlNetModel.from_pretrained(path, local_files_only=True)
        controlnet_pipes[model_name] = StableDiffusionControlNetInpaintPipeline(
            vae=pipe.vae,
            text_encoder=pipe.text_encoder,
            tokenizer=pipe.tokenizer,
            unet=pipe.unet,
            scheduler=pipe.scheduler,
            safety_checker=None,
            feature_extractor=pipe.feature_extractor,
            controlnet=controlnet
        )

    output_dirs = {
        model_name: os.path.join(results_dir, f"controlnet_{model_name}")
        for model_name in controlnet_pipes
    }
    for d in output_dirs.values():
        os.makedirs(d, exist_ok=True)

    prompt = (
        "Replace the masked region with a natural extension of the surrounding background, "
        "ensuring the textures, colors, and lighting blend seamlessly. "
        "Do not recreate any specific object shapes from the mask."
    )

    evaluation_results = []
    all_folders = sorted(os.listdir(base_dir))

    for folder_name in tqdm(all_folders, desc="Processing folders"):
        folder_path = os.path.join(base_dir, folder_name)
        if not os.path.isdir(folder_path) or folder_name == "inpainting_results":
            continue
        print(f"📂 Entering folder: {folder_name}")
        base_id = folder_name
        image_path = os.path.join(folder_path, f"{base_id}_orig.jpg")
        mask_path = os.path.join(folder_path, f"{base_id}_fake_mask.png")
        fake_path = os.path.join(folder_path, f"{base_id}_fake.jpg")

        if not (os.path.exists(image_path) and os.path.exists(mask_path) and os.path.exists(fake_path)):
            print(f"Skipping {base_id}: missing one of the required images.")
            continue

        metrics = {'filename': f"{base_id}_orig.jpg", 'base_id': base_id}
        reference_images = None

        for model_name, model_pipe in controlnet_pipes.items():
            output_path = os.path.join(output_dirs[model_name], f"{base_id}_result_{model_name}.jpg")
            try:
                run_controlnet_inpaint(
                    image_path, mask_path, fake_path,
                    model_pipe, reference_images, prompt, output_path, seed=42
                )

                gt_image = Image.open(image_path).convert("RGB")
                pred_image = Image.open(output_path).convert("RGB")
                psnr, ssim, lpips = evaluate_metrics(gt_image, pred_image)

                metrics[f'{model_name}_PSNR'] = psnr
                metrics[f'{model_name}_SSIM'] = ssim
                metrics[f'{model_name}_LPIPS'] = lpips

            except Exception as e:
                print(f"Error with {model_name} on {base_id}: {str(e)}")
                metrics[f'{model_name}_PSNR'] = -1
                metrics[f'{model_name}_SSIM'] = -1
                metrics[f'{model_name}_LPIPS'] = -1

        evaluation_results.append(metrics)

    csv_file_path = os.path.join(results_dir, "evaluation_results.csv")
    if evaluation_results:
        fieldnames = ['filename', 'base_id'] + [k for k in evaluation_results[0] if k not in ['filename', 'base_id']]
        with open(csv_file_path, mode='w', newline='') as f:
            writer = csv.DictWriter(f, fieldnames=fieldnames)
            writer.writeheader()
            writer.writerows(evaluation_results)

        print(f"\n✅ Done! Results saved to: {csv_file_path}")
    else:
        print("\n❌ No data was processed. Check folder or file naming.")

Mounted at /content/drive


model_index.json:   0%|          | 0.00/548 [00:00<?, ?B/s]

Fetching 16 files:   0%|          | 0/16 [00:00<?, ?it/s]

config.json:   0%|          | 0.00/748 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/4.78k [00:00<?, ?B/s]

...

An error occurred while trying to fetch /root/.cache/huggingface/hub/models--runwayml--stable-diffusion-inpainting/snapshots/8a4288a76071f7280aedbdb3253bdb9e9d5d84bb/unet: Error no file named diffusion_pytorch_model.safetensors found in directory /root/.cache/huggingface/hub/models--runwayml--stable-diffusion-inpainting/snapshots/8a4288a76071f7280aedbdb3253bdb9e9d5d84bb/unet.
Defaulting to unsafe serialization. Pass `allow_pickle=False` to raise an error instead.
An error occurred while trying to fetch /root/.cache/huggingface/hub/models--runwayml--stable-diffusion-inpainting/snapshots/8a4288a76071f7280aedbdb3253bdb9e9d5d84bb/vae: Error no file named diffusion_pytorch_model.safetensors found in directory /root/.cache/huggingface/hub/models--runwayml--stable-diffusion-inpainting/snapshots/8a4288a76071f7280aedbdb3253bdb9e9d5d84bb/vae.
Defaulting to unsafe serialization. Pass `allow_pickle=False` to raise an error instead.
You have disabled the safety checker for <class 'diffusers.pipelin

📂 Entering folder: 00006
Starting inpainting for: 00006_orig.jpg
Resizing images...
Running pipeline...


  0%|          | 0/30 [00:00<?, ?it/s]

Processing folders:   0%|          | 0/41 [17:20<?, ?it/s]


KeyboardInterrupt: 

In [1]:
from google.colab import drive
import os
import csv
from glob import glob
from tqdm import tqdm
from PIL import Image
import torch
from concurrent.futures import ThreadPoolExecutor
from diffusers import StableDiffusionInpaintPipeline, ControlNetModel, StableDiffusionControlNetInpaintPipeline

def evaluate_metrics(gt_img, inpaint_img):
    gt_np = np.array(gt_img).astype(np.float32) / 255.0
    inpaint_np = np.array(inpaint_img).astype(np.float32) / 255.0

    if gt_np.shape != inpaint_np.shape:
        inpaint_img = inpaint_img.resize(gt_img.size, Image.Resampling.LANCZOS)
        inpaint_np = np.array(inpaint_img).astype(np.float32) / 255.0

    psnr = compute_psnr(gt_np, inpaint_np, data_range=1.0)

    min_size = min(gt_np.shape[0], gt_np.shape[1])
    win_size = 7 if min_size >= 7 else (min_size if min_size % 2 == 1 else min_size - 1)
    ssim = compute_ssim(gt_np, inpaint_np, win_size=win_size, channel_axis=2, data_range=1.0)

    gt_tensor = prepare_for_lpips(gt_img)
    inpaint_tensor = prepare_for_lpips(inpaint_img)
    with torch.no_grad():
        lpips_distance = lpips_model(gt_tensor, inpaint_tensor).item()

    return psnr, ssim, lpips_distance

def run_controlnet_inpaint(image_path, mask_path, fake_path, pipe, reference_images, prompt, output_path, seed=42):
    print(f"🖼️ Starting inpainting for: {os.path.basename(image_path)}")

    # Load original images
    image = Image.open(image_path).convert("RGB")
    mask = Image.open(mask_path).convert("L")
    control_image = Image.open(fake_path).convert("RGB")

    # Store original size for later resizing
    original_size = image.size

    # Target resolution for speed-up
    TARGET_RES = (512, 512)
    print("📏 Resizing images to 512x512 for faster processing...")
    image = image.resize(TARGET_RES, Image.Resampling.LANCZOS)
    mask = mask.resize(TARGET_RES, Image.Resampling.LANCZOS)
    control_image = control_image.resize(TARGET_RES, Image.Resampling.LANCZOS)

    generator = torch.manual_seed(seed)

    # Run pipeline
    print("🧠 Running ControlNet pipeline...")
    result = pipe(
        prompt=prompt,
        image=image,
        mask_image=mask,
        control_image=control_image,
        num_inference_steps=15,
        generator=generator
    ).images[0]

    # Optional: Resize back to original resolution
    print("🪄 Resizing result back to original resolution...")
    result = result.resize(original_size, Image.Resampling.LANCZOS)

    # Save output
    print("💾 Saving result...")
    result.save(output_path)
    print("✅ Done with one image.\n")

def process_folder(folder_name):
    folder_path = os.path.join(base_dir, folder_name)
    if not os.path.isdir(folder_path) or folder_name == "inpainting_results":
        return None

    print(f"\n \U0001F4C2 Entering folder: {folder_name}")
    base_id = folder_name
    image_path = os.path.join(folder_path, f"{base_id}_orig.jpg")
    mask_path = os.path.join(folder_path, f"{base_id}_fake_mask.png")
    fake_path = os.path.join(folder_path, f"{base_id}_fake.jpg")

    if not (os.path.exists(image_path) and os.path.exists(mask_path) and os.path.exists(fake_path)):
        print(f"Skipping {base_id}: missing one of the required images.")
        return None

    metrics = {'filename': f"{base_id}_orig.jpg", 'base_id': base_id}
    reference_images = None

    for model_name, model_pipe in controlnet_pipes.items():
        output_path = os.path.join(output_dirs[model_name], f"{base_id}_result_{model_name}.jpg")
        try:
            run_controlnet_inpaint(
                image_path, mask_path, fake_path,
                model_pipe, reference_images, prompt, output_path, seed=42
            )

            gt_image = Image.open(image_path).convert("RGB")
            pred_image = Image.open(output_path).convert("RGB")
            psnr, ssim, lpips = evaluate_metrics(gt_image, pred_image)

            metrics[f'{model_name}_PSNR'] = psnr
            metrics[f'{model_name}_SSIM'] = ssim
            metrics[f'{model_name}_LPIPS'] = lpips

        except Exception as e:
            print(f"Error with {model_name} on {base_id}: {str(e)}")
            metrics[f'{model_name}_PSNR'] = -1
            metrics[f'{model_name}_SSIM'] = -1
            metrics[f'{model_name}_LPIPS'] = -1

    return metrics

In [ ]:
from huggingface_hub import login

if __name__ == "__main__":
    # First try to login to Hugging Face (optional)
    try:
        login(token="hf_YLBEIQWasZiRkWjteKqsSvEjQwtKwlYNoie")  # Only needed if you hit rate limits
    except:
        pass
    drive.mount('/content/drive', force_remount=True)

    base_dir = "/content/drive/MyDrive/dataset_controlnet"
    results_dir = os.path.join(base_dir, "inpainting_results")
    os.makedirs(results_dir, exist_ok=True)

    model_paths = {
        "v11p": "/content/drive/MyDrive/models/controlnet/inpaint",
        "v11f1p": "/content/drive/MyDrive/models/controlnet/seg",
        "v11canny": "/content/drive/MyDrive/models/controlnet/canny"
    }

    pipe = StableDiffusionInpaintPipeline.from_pretrained(
        "runwayml/stable-diffusion-inpainting",
    )

    controlnet_pipes = {}
    for model_name, path in model_paths.items():
        controlnet = ControlNetModel.from_pretrained(path, local_files_only=True)
        controlnet_pipes[model_name] = StableDiffusionControlNetInpaintPipeline(
            vae=pipe.vae,
            text_encoder=pipe.text_encoder,
            tokenizer=pipe.tokenizer,
            unet=pipe.unet,
            scheduler=pipe.scheduler,
            safety_checker=None,
            feature_extractor=pipe.feature_extractor,
            controlnet=controlnet
        )

    output_dirs = {
        model_name: os.path.join(results_dir, f"controlnet_{model_name}")
        for model_name in controlnet_pipes
    }
    for d in output_dirs.values():
        os.makedirs(d, exist_ok=True)

    prompt = (
        "Replace the masked region with a natural extension of the surrounding background, "
        "ensuring the textures, colors, and lighting blend seamlessly. "
        "Do not recreate any specific object shapes from the mask."
    )

    evaluation_results = []
    selected_folders = sorted(os.listdir(base_dir))[:5]  # Only process first 5 folders

    with ThreadPoolExecutor(max_workers=3) as executor:
        for result in tqdm(executor.map(process_folder, selected_folders), total=len(selected_folders)):
            if result:
                evaluation_results.append(result)

    csv_file_path = os.path.join(results_dir, "evaluation_results.csv")
    if evaluation_results:
        fieldnames = ['filename', 'base_id'] + [k for k in evaluation_results[0] if k not in ['filename', 'base_id']]
        with open(csv_file_path, mode='w', newline='') as f:
            writer = csv.DictWriter(f, fieldnames=fieldnames)
            writer.writeheader()
            writer.writerows(evaluation_results)

        print(f"\n✅ Done! Results saved to: {csv_file_path}")
    else:
        print("\n❌ No data was processed. Check folder or file naming.")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Mounted at /content/drive


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

An error occurred while trying to fetch /root/.cache/huggingface/hub/models--runwayml--stable-diffusion-inpainting/snapshots/8a4288a76071f7280aedbdb3253bdb9e9d5d84bb/vae: Error no file named diffusion_pytorch_model.safetensors found in directory /root/.cache/huggingface/hub/models--runwayml--stable-diffusion-inpainting/snapshots/8a4288a76071f7280aedbdb3253bdb9e9d5d84bb/vae.
Defaulting to unsafe serialization. Pass `allow_pickle=False` to raise an error instead.
An error occurred while trying to fetch /root/.cache/huggingface/hub/models--runwayml--stable-diffusion-inpainting/snapshots/8a4288a76071f7280aedbdb3253bdb9e9d5d84bb/unet: Error no file named diffusion_pytorch_model.safetensors found in directory /root/.cache/huggingface/hub/models--runwayml--stable-diffusion-inpainting/snapshots/8a4288a76071f7280aedbdb3253bdb9e9d5d84bb/unet.
Defaulting to unsafe serialization. Pass `allow_pickle=False` to raise an error instead.
You have disabled the safety checker for <class 'diffusers.pipelin


 📂 Entering folder: 00006

 📂 Entering folder: 00009

 📂 Entering folder: 00010


  0%|          | 0/5 [00:00<?, ?it/s]

Skipping 00010: missing one of the required images.
🖼️ Starting inpainting for: 00006_orig.jpg

 📂 Entering folder: 00011
🖼️ Starting inpainting for: 00009_orig.jpg
🖼️ Starting inpainting for: 00011_orig.jpg
📏 Resizing images to 512x512 for faster processing...
📏 Resizing images to 512x512 for faster processing...
📏 Resizing images to 512x512 for faster processing...
🧠 Running ControlNet pipeline...
🧠 Running ControlNet pipeline...
🧠 Running ControlNet pipeline...
